In [1]:
# allow the notebook to access the parent directory so we can import the other modules
# https://stackoverflow.com/a/35273613
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Data Preparation
-----

### Constants and Folder Paths

In [2]:
import os
dataset_folder_path = os.path.join("..", "files", "dataset")
NUM_SAMPLES = 50

### Load Data and Split into *Test*, *Train/Valid*

In [3]:
from data.DataSet import DataSet
dataset = DataSet()
# dataset.load(dataset_folder_path, test_set_percentage=0.3333, validation_set_percentage=0.3333)
dataset.load(dataset_folder_path, test_set_percentage=0, validation_set_percentage=0)

In [4]:
print(len(dataset.train_data))
print(len(dataset.test_data))

3600
0


### Data Preprocessing

In [5]:
from utils.preprocessing import *
from functools import partial
dataset.apply(apply_mean_centering)
dataset.apply(apply_unit_distance_normalization)
#dataset.apply(partial(normalize_pressure_value, max_pressure_val=512))
dataset.apply(partial(spline_interpolate_and_resample, num_samples=NUM_SAMPLES))
dataset.expand(reverse_digit_sequence)
# dataset.apply(lambda digit: convert_xy_to_derivative(digit, normalize=False))
#dataset.apply(partial(convert_xy_to_derivative, normalize=True))

In [6]:
print(len(dataset.train_data))
print(len(dataset.test_data))

7200
0


### Setup Dataset, don't split, don't onehot encode, since we will perform cross validation

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train = np.array(dataset.train_data)
# X_valid = np.array(dataset.valid_data)
# X_test = np.array(dataset.test_data)

Y_train = np.array(dataset.train_labels)
# Convert labels to numpy array and OneHot encode them
# encoder, Y_train, Y_valid, Y_test = dataset.onehot_encode_labels()
# Y_train = Y_train.astype('float32').todense()
# Y_valid = Y_valid.astype('float32').todense()
# Y_test = Y_test.astype('float32').todense()

In [8]:
Y_train.shape

(7200,)

----------
# **Regularized Naive Deep GRU**
----------

In [11]:
PARAM_NUM_EPOCHS = 40
PARAM_BATCH_SIZE = 300

from models.regularized_deep_gru import NaiveRegularizedDeepGRU
from utils.evaluation import cross_validate_model

mymodel = NaiveRegularizedDeepGRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3)

Using TensorFlow backend.



....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/40
4800/4800 [==============================] - 4s 897us/step - loss: 2.1520 - categorical_accuracy: 0.2025 - val_loss: 2.0329 - val_categorical_accuracy: 0.2250
Epoch 2/40
4800/4800 [==============================] - 3s 677us/step - loss: 1.7538 - categorical_accuracy: 0.3471 - val_loss: 1.5116 - val_categorical_accuracy: 0.4921
Epoch 3/40
4800/4800 [==============================] - 3s 679us/step - loss: 1.2495 - categorical_accuracy: 0.5623 - val_loss: 1.1988 - val_categorical_accuracy: 0.6183
Epoch 4/40
4800/4800 [==============================] - 3s 678us/step - loss: 0.8755 - categorical_accuracy: 0.7067 - val_loss: 0.7724 - val_categorical_accuracy: 0.7738
Epoch 5/40
4800/4800 [==============================] - 3s 680us/step - loss: 0.5751 - categorical_accuracy: 0.8146 - val_loss: 0.6025 - val_categorical_accuracy: 0.8279
Epoch 6/40
4800/4800 [==========

4800/4800 [==============================] - 3s 683us/step - loss: 0.3555 - categorical_accuracy: 0.8927 - val_loss: 0.3581 - val_categorical_accuracy: 0.8983
Epoch 8/40
4800/4800 [==============================] - 3s 687us/step - loss: 0.2780 - categorical_accuracy: 0.9148 - val_loss: 0.3340 - val_categorical_accuracy: 0.9088
Epoch 9/40
4800/4800 [==============================] - 3s 688us/step - loss: 0.2108 - categorical_accuracy: 0.9390 - val_loss: 0.2641 - val_categorical_accuracy: 0.9250
Epoch 10/40
4800/4800 [==============================] - 3s 687us/step - loss: 0.1749 - categorical_accuracy: 0.9523 - val_loss: 0.2594 - val_categorical_accuracy: 0.9271
Epoch 11/40
4800/4800 [==============================] - 3s 689us/step - loss: 0.1432 - categorical_accuracy: 0.9623 - val_loss: 0.2361 - val_categorical_accuracy: 0.9317
Epoch 12/40
4800/4800 [==============================] - 3s 691us/step - loss: 0.1326 - categorical_accuracy: 0.9635 - val_loss: 0.2262 - val_categorical_accur

Epoch 14/40
4800/4800 [==============================] - 3s 673us/step - loss: 0.0802 - categorical_accuracy: 0.9806 - val_loss: 0.3810 - val_categorical_accuracy: 0.9037
Epoch 15/40
4800/4800 [==============================] - 3s 670us/step - loss: 0.0784 - categorical_accuracy: 0.9788 - val_loss: 0.3723 - val_categorical_accuracy: 0.9083
Epoch 16/40
4800/4800 [==============================] - 3s 671us/step - loss: 0.0638 - categorical_accuracy: 0.9844 - val_loss: 0.3785 - val_categorical_accuracy: 0.9050
Epoch 17/40
4800/4800 [==============================] - 3s 671us/step - loss: 0.0681 - categorical_accuracy: 0.9835 - val_loss: 0.3618 - val_categorical_accuracy: 0.9117
Epoch 18/40
4800/4800 [==============================] - 3s 673us/step - loss: 0.0632 - categorical_accuracy: 0.9817 - val_loss: 0.3760 - val_categorical_accuracy: 0.9108
Epoch 19/40
4800/4800 [==============================] - 3s 697us/step - loss: 0.0526 - categorical_accuracy: 0.9869 - val_loss: 0.4071 - val_cat

[95.0, 95.25, 91.25]

#### increased batch size

In [ ]:
PARAM_NUM_EPOCHS = 40
PARAM_BATCH_SIZE = 600

from models.regularized_deep_gru import NaiveRegularizedDeepGRU
from utils.evaluation import cross_validate_model

mymodel = NaiveRegularizedDeepGRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

cross_validate_model(X_train, Y_train, mymodel, 3)


....................
Cross validation fold [1]
....................

Train on 4800 samples, validate on 2400 samples
Epoch 1/40
4800/4800 [==============================] - 4s 766us/step - loss: 2.2194 - categorical_accuracy: 0.1829 - val_loss: 2.1009 - val_categorical_accuracy: 0.2496
Epoch 2/40
4800/4800 [==============================] - 3s 582us/step - loss: 2.0138 - categorical_accuracy: 0.2492 - val_loss: 1.8176 - val_categorical_accuracy: 0.3242
Epoch 3/40
4800/4800 [==============================] - 3s 563us/step - loss: 1.6939 - categorical_accuracy: 0.3723 - val_loss: 1.5784 - val_categorical_accuracy: 0.4204
Epoch 4/40
4800/4800 [==============================] - 3s 615us/step - loss: 1.4508 - categorical_accuracy: 0.4773 - val_loss: 1.3909 - val_categorical_accuracy: 0.4958
Epoch 5/40
4800/4800 [==============================] - 3s 571us/step - loss: 1.1906 - categorical_accuracy: 0.5783 - val_loss: 1.0936 - val_categorical_accuracy: 0.6638
Epoch 6/40
4800/4800 [==========